In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
dataset=pd.read_csv(r"/kaggle/input/journal-education-datasets-v000/education_data_for_all_train.csv")
dataset.head()


In [ ]:
random_state = 96

In [ ]:
import pandas as pd
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn import svm

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_curve, roc_auc_score


from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import VotingClassifier


print("\n lightgbm \n","==="*30)
roc_curve_data = {}
auc_scores = {}
algorithms_performances_new = { }

import lightgbm
from sklearn.metrics import confusion_matrix
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from imblearn.metrics import geometric_mean_score
import lightgbm as lgb
import joblib



###### print("===========***"*3," plot functions ","===========***"*3)
import matplotlib.pyplot as plt

def plot_roc_curve(fpr_list, tpr_list, pic_name_list,label=None):
    
    fpr = fpr_list[0]
    tpr = tpr_list[0]
    
    fpr_1 = fpr_list[1]
    tpr_1 = tpr_list[1]
    
    fpr_2 = fpr_list[2]
    tpr_2 = tpr_list[2]
    
    pic_name   =  "AUC : " + str(pic_name_list[0])
    pic_name_1 =  "AUC : " + str(pic_name_list[1])
    pic_name_2 =  "AUC : " + str(pic_name_list[2])
    
    pic_route = r"\kaggle\working\ " + pic_name + ".png"
    
    fig, ax = plt.subplots(figsize=(7,7)) 
    
#     plt.figure(figsize=(5,5))
    ax.plot(fpr  , tpr  , linewidth=3, color = "cornflowerblue", label=pic_name)
    ax.plot(fpr_1, tpr_1, linewidth=3, color = "darkorange", label=pic_name_1)
    ax.plot(fpr_2, tpr_2, linewidth=3, color = "limegreen", label=pic_name_2)
    
#     plt.plot(fpr, tpr, linewidth=2, label=pic_name)
    
    ax.plot([0,1],[0,1], "r--") 
    ax.axis([0,1,0,1])
    ax.legend(loc="lower left")
#     ax.set_title(pic_name)
    ax.grid(False)
    
    fig = ax.figure
    fig.savefig(pic_route)

    ax.set_xlabel("False Positive Rate")
    ax.set_ylabel("True Positive rate")

    
    
    
print("===========***"*3," metric function ","===========***"*3)


def performances_metrics(algorithms_performances_new, 
                         auc_avg,auc_std,
                         acc_avg,acc_std,
                         f1_avg,f1_std,
                         key_words,
                         y_test,
                         y_pred):
    
    

    key_words = str(key_words)
    print (" in the function, the key words == ",key_words )
    print("std_auc",std_auc)
    print("=========",key_words,"====@@====",auc_avg, "±" , auc_std ) #roc_auc_score(y_test, y_pred))

    algorithms_performances_new[key_words] = [auc_avg] # [roc_auc_score(y_test, y_pred)]
    
    cm = confusion_matrix(y_test, output_y_pred(y_pred))
 
    cm = cm.astype(np.float32)
    FP = cm.sum(axis=0) - np.diag(cm)
    FN = cm.sum(axis=1) - np.diag(cm)
    TP = np.diag(cm)
    TN = cm.sum() - (FP + FN + TP)
    
    TPR = TP / (TP + FN)     # Sensitivity, hit rate, recall, or true positive rate
    TNR = TN / (TN + FP)     # Specificity or true negative rate
    PPV = TP / (TP + FP)     # Precision or positive predictive value
    NPV = TN / (TN + FN)     # Negative predictive value
    FPR = FP / (FP + TN)     # Fall out or false positive rate 
    FNR = FN / (TP + FN)     # False negative rate
    FDR = FP / (TP + FP)     # False discovery rate
    
    fpr, tpr, thresholds = roc_curve(y_test, y_pred)
    
    # accuracy    = accuracy_score(y_test, output_y_pred(y_pred))
    precision   = precision_score(y_test, output_y_pred(y_pred))
    recall      = recall_score(y_test, output_y_pred(y_pred))
    # f1          = f1_score(y_test,output_y_pred(y_pred))
    KS          = max(TPR-FPR)
    Specificity = 1- fpr
    G_Mean      = geometric_mean_score(y_test, output_y_pred(y_pred))
    BAcc        = ((TPR + TNR)/2)[0]                         #balanced accuracy (BAcc)
    tpr         = tpr
    MCC         = (TP*TN-FP*FN)/((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))**0.5
#     precision = TP/(TP+FP)
#     recall    = TP/(TP+FN)
    counts_ratio    = output_y_pred(y_pred).sum()/len(y_pred)
    Randon_accuracy = ((TP + FN)*(TP+FP) + (TN+FP)*(TN+FN))/(TP+TN+FP+FN)**2
    kappa0          = (TP+TN)/(TP+TN+FP+FN)
    kappa           = (kappa0-Randon_accuracy)/(1-Randon_accuracy)                                                     
                                                         

    
    algorithms_performances_new[key_words].append(auc_std) 
    algorithms_performances_new[key_words].append(acc_avg)
    algorithms_performances_new[key_words].append(acc_std)
    algorithms_performances_new[key_words].append(f1_avg)
    algorithms_performances_new[key_words].append(f1_std)
    
    
    algorithms_performances_new[key_words].append(precision)
    algorithms_performances_new[key_words].append(recall)
    # algorithms_performances_new[key_words].append(f1)
    algorithms_performances_new[key_words].append(KS)
    algorithms_performances_new[key_words].append(G_Mean)
    algorithms_performances_new[key_words].append(BAcc)
    algorithms_performances_new[key_words].append(tpr[1])
    algorithms_performances_new[key_words].append(MCC[1])
    algorithms_performances_new[key_words].append(counts_ratio)
    algorithms_performances_new[key_words].append(kappa[1])
    
    pd.DataFrame(algorithms_performances_new,index=["AUC","AUC std",
                                                    "Accuracy","Accuracy std",
                                                    "F1 score","F1 score std",
                                                    "Precision","Recall",
                                                    "KS","G_Mean","BAcc","tpr","MCC","counts_ratio","kappa"
                                                   ]).transpose()
    return algorithms_performances_new


def output_y_pred(y_pred):
    threshold = 0.5
    y_pred[y_pred <= threshold] = 0.
    y_pred[y_pred >  threshold] = 1.
    return y_pred


def data_preprocessing(df):
    
    df.rename(columns={'Target':'label'},inplace = True)
    df.rename(columns = {'Nacionality':'Nationality', 'Age at enrollment':'Age'}, inplace = True)
    
    return df


def standardize_features(df, label_column):
    # Separate features and label
    features = df.drop(columns=label_column)
    label = df[label_column]

    # Initialize the StandardScaler
    scaler = StandardScaler()

    # Fit and transform the features
    features_scaled = pd.DataFrame(scaler.fit_transform(features), columns=features.columns)

    # Combine the scaled features with the label
    df_scaled = pd.concat([features_scaled, label.reset_index(drop=True)], axis=1)

    return df_scaled


In [ ]:
import warnings

warnings.filterwarnings('ignore')

import pandas as pd
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn import svm

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_curve, roc_auc_score


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

In [ ]:
# !pip install ctgan

In [ ]:
!pip install sdv

In [ ]:
!git clone https://github.com/penfever/TuneTables.git
# !rm -rf /kaggle/working/*

In [ ]:
cd /kaggle/working/TuneTables/tunetables

In [ ]:
! python batch/run_tt_job.py
# ! python  run_baselines.py

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score,\
                            accuracy_score, balanced_accuracy_score,classification_report,confusion_matrix#,\
#                             plot_confusion_matrix, confusion_matrix


# import sklearn.metrics.plot_confusion_matrix


from sklearn.model_selection import KFold, GridSearchCV

import lightgbm as lgb
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, Concatenate
from tensorflow.keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import RandomNormal
import tensorflow.keras.backend as K
from sklearn.utils import shuffle

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
# np.random.seed(1635848)

In [ ]:
# df_for_train            = pd.read_csv(r'/kaggle/input/journal-education-datasets-v000/education_data_for_all_train.csv')
# df_for_train = data_preprocessing(df_for_train)
# df_for_train.drop(["Unnamed: 0"],axis = 1,inplace = True)


df_for_train            = pd.read_csv(r'/kaggle/input/journal-education-datasets-v000/education_data_for_all_train_original labels.csv')
df_for_train = data_preprocessing(df_for_train)
df_for_train.drop(["Unnamed: 0"],axis = 1,inplace = True)


df_for_test             = pd.read_csv(r'/kaggle/input/journal-education-datasets-v000/education_data_for_all_test.csv')
df_for_test = data_preprocessing(df_for_test)
df_for_test.drop(["Unnamed: 0"],axis = 1,inplace = True)

# df_for_train_distilled  = pd.read_excel(r'/kaggle/input/journal-education-datasets-v000/distilled_for_train_TKP.xlsx')
df_for_train_distilled  = pd.read_excel(r'/kaggle/input/journal-education-datasets-v000/distilled_for_train.xlsx')
df_for_train_distilled = data_preprocessing(df_for_train_distilled)
df_for_train_distilled.drop(["Unnamed: 0"],axis = 1,inplace = True)
df_for_train_distilled.columns = df_for_train.columns



In [ ]:

df_for_train['label'] = df_for_train['label'].map({
                                    'Dropout':1,
                                    'Enrolled':0,
                                    'Graduate':0})

# df_for_train.head()

In [ ]:
len_datashape = df_for_train.shape[0]
# len_datashape

In [ ]:

# df.head()
# df = df.iloc[:-10,:]
print(df_for_train.shape)

# df_original = data_preprocessing(df_for_train)

scaler = StandardScaler()

# df_original['label'] = df_original['label'].map({'Dropout':1,
#                                                 'Enrolled':0,
#                                                 'Graduate':0})
# print(df_original["label"].unique())

# X = scaler.fit_transform(df_original.drop('label', axis = 1))
# y = df_original['label'].values

# X_train_original, X_test_original, y_train_original, y_test_original = train_test_split(X, y, test_size=0.2,
#                                                                                         random_state = random_state)

# df_train_original = np.concatenate ((X_train_original,y_train_original.reshape(-1,1)),axis=1)
# df_test_original  = np.concatenate ((X_test_original,y_test_original.reshape(-1,1)),axis=1)


discrete_columns = list(df_for_train.columns.values)

df_for_train_scaled = standardize_features(df_for_train, 'label')
df_for_test_scaled  = standardize_features(df_for_test, 'label')
df_for_train_distilled_scaled  = standardize_features(df_for_train_distilled, 'label')


X_train_original = df_for_train_scaled.iloc[:,:-1].values   
y_train_original = df_for_train_scaled["label"].values

# X_test_original  # 这是general的测试集 X_test
# y_test_original  # 这是general的测试集 y_test

X_test_original  = df_for_test_scaled.iloc[:,:-1].values
y_test_original  = df_for_test_scaled["label"].values


X_train_distilled = df_for_train_distilled_scaled.iloc[:,:-1].values  
y_train_distilled = df_for_train_distilled_scaled["label"].values

In [ ]:
# df_for_train_distilled_scaled.head()

In [ ]:
# df_for_train_scaled.head()

In [ ]:
# df_for_test_scaled.head()

In [ ]:
# df_the_distilled_100.columns

In [ ]:

''' this is the distilled dataset ''' #数据蒸馏的样本数据集

df_x_distilled  = df_for_train_distilled.drop(["label"],axis = "columns")
df_y_distilled  = df_for_train_distilled["label"]


df_labelled_x   = df_for_train.drop(["label"],axis = "columns")
df_labelled_y   = df_for_train["label"]

In [ ]:
# df_for_train_distilled.head()

In [ ]:
print("data synthetic process started...")


epochs = [10,100,500,1000,2000,3000,5000]
epochs = [100,500,2000,len_datashape]
epoch  = epochs[2]
epoch  = epochs[1]
    

def the_conditional_generated_datasets(df_for_train_scaled,epoch):
    
    df_train_shaped = df_for_train_scaled

    import time

    import sdv
    from sdv.metadata import SingleTableMetadata
    from sdv.single_table import GaussianCopulaSynthesizer
    
    from sdv.single_table import CTGANSynthesizer

    '''training the conditional gannetwork'''
    metadata = SingleTableMetadata()
    metadata.detect_from_dataframe(df_train_shaped)
    metadata.visualize(show_table_details='summarized',output_filepath='my_metadata.png')

    '''generate the synthetic data'''



    start_time = time.time()

    synthesizer = CTGANSynthesizer(metadata,enforce_rounding=False,
                                   epochs=epoch, 
                                   verbose=True)
    
    synthesizer.fit(df_train_shaped)
    synthetic_data = synthesizer.sample(num_rows=1000)

    end_time = time.time()

    print("running time == ", end_time - start_time )

    ''' generating the datasets'''
    synthetic_data_dict = {}
    sample_sizes = [100,500,1000,2000,3000]

    # Create synthetic data
    for sample_size in sample_sizes:
        key_words = "syn_data_set_" + str(sample_size)
        synthetic_data_dict[key_words] = synthesizer.sample(sample_size)
        synthetic_data_dict[key_words].to_excel("/kaggle/working/stats_"+ key_words + "_" + str(epoch)+ "_.xlsx")

    ''' plot the conditional tabular generative adversal network'''

    def synthetic_plot(synthesizer):
        G_Loss = synthesizer.get_loss_values()["Generator Loss"]
        D_Loss = synthesizer.get_loss_values()["Discriminator Loss"]
        x_index = synthesizer.get_loss_values()["Epoch"]

        fig,ax = plt.subplots() 
        ax.plot(x_index,G_Loss,label='G_Loss')
        ax.plot(x_index,D_Loss,label='D_Loss')

        ax.set_xlabel('epochs')
        ax.set_ylabel('L')

        ax.set_title('synthetic with {} epoches'.format(epoch))
        ax.legend()
        plt.show()

    synthetic_plot(synthesizer)
    
    return synthetic_data_dict

# epoch = 200
synthetic_data_dict = the_conditional_generated_datasets(df_for_train_scaled,epoch)


                                                         
synthetic_data_dict["syn_data_set_2000"].rename(columns={'Target':'label'},inplace = True)

In [ ]:
synthetic_data_dict["syn_data_set_2000"].to_csv('/kaggle/working/synthetic_2000.csv')

In [ ]:
y_train_original[:5]

In [ ]:
# Import libraries
import os
import pathlib

# Let's say you already have list of files under /kaggle/working directory
for n in range(10):
    with open(os.getcwd() + '/file_' + str(n) + '.png', 'w') as file:
        file.write('delete this file')
        
# Deleting the files
files_to_delete = './*.png' # this considers only ".txt" files. If you want to delete all files, use "./*"
files_list = pathlib.Path(os.getcwd()).glob(files_to_delete)
for file_path in files_list:
    os.remove(file_path)

In [ ]:
# df_original = pd.read_csv(r"/kaggle/input/higher-education-predictors-of-student-retention/dataset.csv")
# # df.info()
# df_original.rename(columns = {'Nacionality':'Nationality', 'Age at enrollment':'Age'}, inplace = True)

# df_original.rename(columns={'Target':'label'},inplace = True)
# df_original.label.value_counts()
# scaler = StandardScaler()
# df_original['label'] = df_original['label'].map({
#                                     'Dropout':1,
#                                     'Enrolled':0,
#                                     'Graduate':0})


df_synthetic_2000 = synthetic_data_dict["syn_data_set_2000"].copy()
# df_synthetic_2000.rename(columns = {'Nacionality':'Nationality', 'Age at enrollment':'Age'}, inplace = True)

# df_synthetic_2000.rename(columns={'Target':'label'},inplace = True)

# scaler = StandardScaler()
# df_synthetic_2000['label'] =df_synthetic_2000['label'].map({
#                                         'Dropout':1,
#                                         'Enrolled':0,
#                                         'Graduate':0})


In [ ]:
import time
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# dataset_dict= {}

# df_labelled_x_completee_data  = df_selective_2.drop(["Target"],axis = "columns")
# df_labelled_x_selective_data = df_selective_2[important_cols]

# dataset_dict["df_labelled_x_completee_data"] = df_labelled_x_completee_data
# dataset_dict["df_labelled_x_selective_data"] = df_labelled_x_selective_data

# df_y = df_selective_2["Target"]



# sns.set(color_codes=True)
color_dict = dict({'0':'orange',
                   '1':'#067ef7'
                   })

nrows = 7

def original_data_hist_plot(dataset_dict):
    
    sns.set_theme(style="white")
    
    n = len(dataset_dict)
    
    fig, ax = plt.subplots(nrows,int(n/nrows), sharey="row",figsize=(10,10))

    ck = 0
    keys = list(dataset_dict.keys())
    for keys_index in range(len(dataset_dict.keys())):
        
        X_CV =  dataset_dict[keys[keys_index]]
        y_CV =  df_y
    
#         print("imbalanced ratio : ",len(y_give_CV)/y_give_CV.sum())
        X_stand = StandardScaler().fit_transform(X_CV)
        pca = PCA(n_components=2).fit(X_stand)
        X_pca = pca.transform(X_stand)
        
            
#         fig_name = "ratio : " + str(np.round(len(y_CV)/y_CV.sum(),1))

    
        sns.scatterplot(x=X_pca[:, 0], 
                        y=X_pca[:, 1], hue=y_CV.astype(str), alpha=0.7, ax = ax[ck//nrows,ck%nrows],
                        edgecolor="white",
                        palette = color_dict,edgecolors=None)
        
        
#         plt.xlabel('1st Principal Component')
#         plt.ylabel('2nd Principal Component')
        
        ax[ck//nrows,ck%nrows].title.set_text(keys[keys_index])
        ax[ck//nrows,ck%nrows].grid(False)
        ax[ck//nrows,ck%nrows].set_xlim(-10, 20)
        ax[ck//nrows,ck%nrows].set_ylim(-5, 20)

        
        ck += 1

        plt.tick_params(bottom=False, labelbottom=False, labelleft=False)


    plt.grid(False)
    plt.suptitle('Training set visualisation using PCA features')
    plt.savefig("data distribution plotting.png",dpi=600)
    plt.show()

In [ ]:

# Histogram

selected_cols = ["Curricular units 2nd sem (grade)","Curricular units 2nd sem (approved)",
                "Curricular units 1st sem (grade)","Curricular units 1st sem (approved)",
                "Gender","Age",
                 "Unemployment rate","Debtor"]

ck = 0
nrows = 2
ncols = int(len(selected_cols)/nrows)


In [ ]:
import seaborn

''' this is the first one '''   #原始数据集


# print("\n原始训练集：")
# print(df_for_train_scaled.shape)
# print("\n原始测试集：")
# print(df_for_test_scaled.shape)

# df_original_y = {}
# df_original_x = X_train_original
# df_original_y = y_train_original




original_column_names = list(df_for_train.columns)
'''============================================'''
'''============================================'''


''' this is the second one '''  # 2000个样本的数据集
df_synthetic_2000_students  = df_synthetic_2000.copy()

df_synthetic_2000_students_x  = df_synthetic_2000_students.drop(["label"],axis = "columns")
df_synthetic_2000_students_y  = df_synthetic_2000_students["label"]

'''============================================'''
'''============================================'''





In [ ]:
# df_for_train_scaled.head()

In [ ]:
df_x_original_for_train = X_train_original
df_y_original_for_train = y_train_original

df_x_synthetic_for_train   = df_synthetic_2000_students.drop(["label"],axis = "columns")
df_y_synthetic__for_train  = df_synthetic_2000_students["label"]


df_y = {}


important_cols = [
                   'Marital status', 'Application mode', 'Application order', 'Course',
                   'Daytime/evening attendance', 'Previous qualification', 'Nationality',
                   "Mother's qualification", "Father's qualification","Mother's occupation", "Father's occupation", 
                   'Displaced',
                   'Educational special needs', 
                   'Debtor', 'Tuition fees up to date',
                   
                   'Gender', 'Scholarship holder', 'Age', 'International',
                   
#                    'Curricular units 1st sem (credited)',
#                    'Curricular units 1st sem (enrolled)',
#                    'Curricular units 1st sem (evaluations)',
#                    'Curricular units 1st sem (approved)',
#                    'Curricular units 1st sem (grade)',
#                    'Curricular units 1st sem (without evaluations)',
#                    'Curricular units 2nd sem (credited)',
#                    'Curricular units 2nd sem (enrolled)',
#                    'Curricular units 2nd sem (evaluations)',
#                    'Curricular units 2nd sem (approved)',
#                    'Curricular units 2nd sem (grade)',
#                    'Curricular units 2nd sem (without evaluations)',

                   'Unemployment rate','Inflation rate', 'GDP', 
#                    'Target'
                   ]

complete_cols = [   
                   # demographic
                   'Marital status', 'Application mode', 'Application order', 'Course',
                   'Daytime/evening attendance', 'Previous qualification', 'Nationality',
                   "Mother's qualification", "Father's qualification","Mother's occupation", "Father's occupation", 
                   'Displaced',
                   'Educational special needs', 
                     
                   # socioeconomic
                   'Debtor', 'Tuition fees up to date','Scholarship holder', 'International', 
                   
                   'Gender',  'Age',
                   
                   'Curricular units 1st sem (credited)',
                   'Curricular units 1st sem (enrolled)',
                   'Curricular units 1st sem (evaluations)',
                   'Curricular units 1st sem (approved)',
                   'Curricular units 1st sem (grade)',
                   'Curricular units 1st sem (without evaluations)',
                   'Curricular units 2nd sem (credited)',
                   'Curricular units 2nd sem (enrolled)',
                   'Curricular units 2nd sem (evaluations)',
                   'Curricular units 2nd sem (approved)',
                   'Curricular units 2nd sem (grade)',
                   'Curricular units 2nd sem (without evaluations)',
                    
                    # macroeconomic
                   'Unemployment rate','Inflation rate', 'GDP', 
#                    'Target'
                   ]

academic_cols = [
                   'Curricular units 1st sem (credited)',
                   'Curricular units 1st sem (enrolled)',
                   'Curricular units 1st sem (evaluations)',
                   'Curricular units 1st sem (approved)',
                   'Curricular units 1st sem (grade)',
                   'Curricular units 1st sem (without evaluations)',
                   'Curricular units 2nd sem (credited)',
                   'Curricular units 2nd sem (enrolled)',
                   'Curricular units 2nd sem (evaluations)',
                   'Curricular units 2nd sem (approved)',
                   'Curricular units 2nd sem (grade)',
                   'Curricular units 2nd sem (without evaluations)',
                    ]



Macroeconomic_col = ['Unemployment rate',
                     'Inflation rate',
                     'GDP']

Socioeconomic_col = ['Debtor', 
                     'Tuition fees up to date',
                     'Scholarship holder',
                     'International', ]
Demographic_col   = [                   
                       'Marital status', 'Application mode', 'Application order', 'Course',
                       'Daytime/evening attendance', 'Previous qualification', 'Nationality',
                       "Mother's qualification", "Father's qualification","Mother's occupation", "Father's occupation", 
                       'Displaced',
                       'Educational special needs', ]


df_x_original_complete      = df_for_train_scaled[complete_cols]

df_x_original_academic      = df_for_train_scaled[academic_cols]

df_x_original_macro_econ    = df_for_train_scaled[Macroeconomic_col]

df_x_original_Demographic   = df_for_train_scaled[Demographic_col]

df_x_original_Socioeconomic = df_for_train_scaled[Socioeconomic_col]

df_x_original_onlynot_academic = df_for_train_scaled[important_cols]



df_dataset_dict = {}

df_dataset_dict["complete data"]             = df_x_original_complete
df_dataset_dict["only academic data"]        = df_x_original_academic
df_dataset_dict["without academic data"]     = df_x_original_onlynot_academic
df_dataset_dict[" only Demographic data"]    = df_x_original_Demographic


df_y["complete data"]            = df_y_original_for_train
df_y["only academic data"]       = df_y_original_for_train
df_y["without academic data"]    = df_y_original_for_train
df_y[" only Demographic data"]   = df_y_original_for_train

# df_labelled_dict["academic_cols"]   =

plot2 = {}
plot2["complete data"       ]     = df_x_original_complete
plot2["only academic data"   ]     = df_x_original_academic
plot2["without academic data"]      = df_x_original_onlynot_academic
# plot2[" only macro_econ data"]    = df_labelled_x_macro_econ
plot2[" only Demographic data"]     = df_x_original_Demographic
# plot2[" only Socioeconomic data"]    = df_labelled_x_Socioeconomic


'''→→→→→→→→→→→   ←←←←←←←←←←←←←'''
''' this is for the generated adversal network'''
'''→→→→→→→→→→→   ←←←←←←←←←←←←←'''

df_x_synthetic_for_train_complete      = df_x_synthetic_for_train[complete_cols]

df_x_synthetic_for_train_academic      = df_x_synthetic_for_train[academic_cols]

df_x_synthetic_for_train_macro_econ    = df_x_synthetic_for_train[Macroeconomic_col]

df_x_synthetic_for_train_Demographic   = df_x_synthetic_for_train[Demographic_col]

df_x_synthetic_for_train_Socioeconomic = df_x_synthetic_for_train[Socioeconomic_col]

df_x_synthetic_for_train_onlynot_academic      = df_x_synthetic_for_train[important_cols]




df_dataset_dict["synthetic ==> complete data"]             = df_x_synthetic_for_train_complete
df_dataset_dict["synthetic ==> only academic data"]        = df_x_synthetic_for_train_academic
df_dataset_dict["synthetic ==> without academic data"]     = df_x_synthetic_for_train_onlynot_academic
df_dataset_dict["synthetic ==> only Demographic data"]     = df_x_synthetic_for_train_Demographic


df_y["synthetic ==> complete data"]           = df_synthetic_2000_students_y
df_y["synthetic ==> only academic data"]      = df_synthetic_2000_students_y
df_y["synthetic ==> without academic data"]   = df_synthetic_2000_students_y
df_y["synthetic ==> only Demographic data"]   = df_synthetic_2000_students_y



'''→→→→→→→→→→→   ←←←←←←←←←←←←←'''
''' this is for the distilled datasets'''
'''→→→→→→→→→→→   ←←←←←←←←←←←←←'''

df_labelled_x_distilled_complete      = df_for_train_distilled[complete_cols]

df_labelled_x_distilled_academic      = df_for_train_distilled[academic_cols]

df_labelled_x_distilled_macro_econ    = df_for_train_distilled[Macroeconomic_col]

df_labelled_x_distilled_Demographic   = df_for_train_distilled[Demographic_col]

df_labelled_x_distilled_Socioeconomic = df_for_train_distilled[Socioeconomic_col]

df_labelled_x_distilled_onlynot_academic = df_for_train_distilled[important_cols]




df_dataset_dict["distilled ==> complete data"]             = df_labelled_x_distilled_complete
df_dataset_dict["distilled ==> only academic data"]        = df_labelled_x_distilled_academic
df_dataset_dict["distilled ==> without academic data"]     = df_labelled_x_distilled_onlynot_academic
df_dataset_dict["distilled ==> only Demographic data"]     = df_labelled_x_distilled_Demographic


df_y["distilled ==> complete data"]           = df_y_distilled
df_y["distilled ==> only academic data"]      = df_y_distilled
df_y["distilled ==> without academic data"]   = df_y_distilled
df_y["distilled ==> only Demographic data"]   = df_y_distilled


In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

Neural_network = make_pipeline(
                                StandardScaler(),
                                MLPClassifier(hidden_layer_sizes=(5,),
                                             activation='logistic', 
                                             max_iter=10000,
                                             learning_rate='invscaling',
                                             random_state=0))

In [ ]:
def original_test_dataset_transform(df_test_original,columns):
   
    columns = list(columns)
    columns.append("label")

    # print("columns:",columns)

    df = df_test_original[columns]

    scaler = StandardScaler()

    original_test_x = scaler.fit_transform(df.drop('label', axis = 1))
    original_test_y = df['label'].values

    print("the trainset size == ",original_test_x.shape)
    print("the testset size == " ,original_test_y.shape)

    # print("original_test_y is none", df)
    
    return original_test_x,original_test_y
    

In [ ]:
# Classifier Libraries
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
import collections

import lightgbm
from sklearn.metrics import confusion_matrix
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from imblearn.metrics import geometric_mean_score
import lightgbm as lgb
import joblib

from sklearn.model_selection import train_test_split, cross_val_predict

random_state_number = 92

# print("X_train",X_train.shape)
# print("X_test",X_test.shape)
# print("y_train",y_train.shape)
# print("y_test",y_test.shape)


algorithms_performances = {}
classifiers = {
                "Logisitic"    : LogisticRegression(),
                "KNearest"     : KNeighborsClassifier(),
                "Decision Tree": DecisionTreeClassifier(),
                "Random Forest": RandomForestClassifier(),
                }

model_explanation_linear = {}
df_shap_dict = {}


nsplits = 5

track_datasets = []
# for n in ["normal model","LightGBM","Smote LGBM"]:
for n in ["normal model","LightGBM","Smote LGBM"]:
    if n == "normal model":
        for key, classifier in classifiers.items():
            fpr_list = []
            tpr_list = []
            pic_name_list = []
            for data_set in df_dataset_dict.keys():
                print(data_set)
                track_datasets.append(data_set)

                from sklearn.model_selection import KFold
                from sklearn.metrics import roc_auc_score, roc_curve
                
                kf = KFold(n_splits=nsplits, shuffle=True, random_state=random_state_number)

                X = df_dataset_dict[data_set]   # x for train
                y = df_y[data_set]#.to_numpy()  # y for train
                test_columns = df_dataset_dict[data_set].keys()
                
                auc_scores = []
                acc_scores = []
                f1_scores  = [] 

                
                for fold, (train_index, test_index) in enumerate(kf.split(X)):
                    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
                    y_train, y_test = y[train_index], y[test_index]

                
                    X_test, y_test = original_test_dataset_transform(df_for_test_scaled, test_columns)
                
                    classifier.fit(X_train, y_train)
                    print(f"Fold {fold+1} train size: ", X_train.shape)
                
                    model_explanation_linear[key + "_" + str(data_set) + "_fold_" + str(fold+1)] = classifier
                
                    y_pred = classifier.predict_proba(X_test)[:, 1]
                    fpr, tpr, thresh = roc_curve(y_test, y_pred)
                    auc = roc_auc_score(y_test, y_pred)
                    auc_scores.append(auc)
                
                    print(f"{key} Fold {fold+1} train AUC score:", auc)
                    
                    acc = accuracy_score(y_test, output_y_pred(y_pred))
                    acc_scores.append(acc)

                    f1  = f1_score(y_test,output_y_pred(y_pred))
                    f1_scores.append(f1)
                    
                
                avg_auc = np.mean(auc_scores)
                std_auc = np.std(auc_scores)

                avg_acc = np.mean(acc_scores)
                std_acc = np.std(acc_scores)

                avg_f1 = np.mean(f1_scores)
                std_f1 = np.std(f1_scores)
                
                print(f"Average AUC score across folds: {avg_auc}")
                print(f"Standard deviation of AUC scores: {std_auc}")

                print(f"Average ACC score across folds: {avg_acc}")
                print(f"Standard deviation of ACC scores: {std_acc}")

                print(f"Average F1 score across folds: {avg_f1}")
                print(f"Standard deviation of F1 scores: {std_f1}")
                
                key_words = key + "_" + data_set
                algorithms_performances[key_words] = avg_auc
                
                print("std_auc",std_auc)
                performances_metrics(algorithms_performances,
                                     avg_auc,std_auc,
                                     avg_acc,std_acc,
                                     avg_f1,std_f1,
                                     key_words,
                                     y_test, y_pred)
                
                pic_name = str(key_words) + " " + str(round(auc,3))
                
                pic_name_list.append(pic_name)
                print("this is the pic name list \n", pic_name_list)
                fpr_list.append(fpr)
                tpr_list.append(tpr)

            plot_roc_curve(fpr_list, tpr_list,pic_name_list)

            #===============================================
            dataset_smotes = 0
            
    elif n == "LightGBM":

        model_explanation = {}
        
        learning_rate = 0.1
        params = {'learning_rate': learning_rate,
                  "seed": 25,
                  "num_iterations": 20,
                  'metric': ["auc","cross_entropy"],
                  "objective":"binary",
                   'min_data_in_leaf': 3, 
                  'verbose': -1
                   }
        
        
        fpr_list = []
        tpr_list = []
        pic_name_list = []
        
        for data_set in df_dataset_dict.keys():
            print(data_set)

            kf = KFold(n_splits=nsplits, shuffle=True, random_state=random_state_number)
            
            X = df_dataset_dict[data_set]
            y = df_y[data_set]#.to_numpy()
            test_columns = df_dataset_dict[data_set].keys()
            
            auc_scores = []
            acc_scores = []
            f1_scores  = [] 
            
            for fold, (train_index, test_index) in enumerate(kf.split(X)):
                X_train, X_test = X.iloc[train_index], X.iloc[test_index]
                y_train, y_test = y[train_index], y[test_index]

                
                df_for_shap = X_train.copy()
                df_for_shap["label"] = y_train.tolist()
                df_shap_dict[data_set + "_fold_" + str(fold+1)] = df_for_shap

                # print("df_for_shap ====  types ====",df_for_shap.head())
            
                X_test, y_test = original_test_dataset_transform(df_for_test_scaled, test_columns)
                
                # df_shap = 
                
                clf = lgb.LGBMClassifier(
                    boosting_type='gbdt',
                    num_leaves=50, reg_alpha=0.0, reg_lambda=1,
                    max_depth=-1, n_estimators=100, objective='binary',
                    subsample=0.7, colsample_bytree=0.7, subsample_freq=1,
                    learning_rate=0.05, min_child_weight=50, random_state=2000, n_jobs=100,
                    verbose = -1,
                )
            
                clf.fit(X_train, y_train,
                        eval_set=[(X_train, y_train)],
                        eval_metric='auc',
                        # verbose=-1# early_stopping_rounds=1000
                       )
            
                model_explanation[data_set + "_fold_" + str(fold+1)] = clf



            
                

            
                y_pred = clf.predict_proba(X_test)[:, 1]
            
                y_demonstrate = {}
                y_demonstrate["y_pred"] = y_pred
                y_demonstrate["y_test"] = y_test
            
                fpr_gbc, tpr_gbc, thresh_gbc = roc_curve(y_test, y_pred)
               
                auc = roc_auc_score(y_test, y_pred)
                auc_scores.append(auc)
                
                print(f"Fold {fold+1} train AUC score: {auc}")

                acc = accuracy_score(y_test, output_y_pred(y_pred))
                acc_scores.append(acc)

                f1  = f1_score(y_test,output_y_pred(y_pred))
                f1_scores.append(f1)

            
            avg_auc = np.mean(auc_scores)
            std_auc = np.std(auc_scores)

            avg_acc = np.mean(acc_scores)
            std_acc = np.std(acc_scores)

            avg_f1 = np.mean(f1_scores)
            std_f1 = np.std(f1_scores)


    
            # Update algorithms_performances with the average AUC
            # algorithms_performances[f"LightGBM_{data_set}_avg_auc"] = avg_auc
            
            print(f"Average AUC score across folds: {avg_auc}")
            print(f"Standard deviation of AUC scores: {std_auc}")


            key_words = "LightGBM @@ " + data_set

            print ("??",key_words)
            algorithms_performances[key_words] = avg_auc
            
            performances_metrics(algorithms_performances,
                                     avg_auc,std_auc,
                                     avg_acc,std_acc,
                                     avg_f1,std_f1,
                                     key_words, 
                                     y_test, y_pred)

            pic_name = "LightGBM @@ " + " " +  str(round(auc,3))
            print(" ++++++++++++++++++++++++++++ key_words ,", key_words)

            pic_name_list.append(pic_name)
            fpr_list.append(fpr_gbc)
            tpr_list.append(tpr_gbc)

        plot_roc_curve(fpr_list, tpr_list,pic_name_list)
        
    elif n == "Smote LGBM":
        
        learning_rate = 0.1
        params = {'learning_rate': learning_rate,
                  "seed": 25,
                  "num_iterations": 20,
                  'metric': ["auc","cross_entropy"],
                  "objective":"binary",
                   'min_data_in_leaf': 3, 
                   }
        
        
        fpr_list = []
        tpr_list = []
        pic_name_list = []
        
#         print("smote algorithm \n")      
        from imblearn.over_sampling import SMOTE

        for data_set in df_dataset_dict.keys():
            print(data_set)

            kf = KFold(n_splits=nsplits, shuffle=True, random_state=random_state_number)
            
            X = df_dataset_dict[data_set]
            y = df_y[data_set]#.to_numpy()
            test_columns = df_dataset_dict[data_set].keys()
            
            auc_scores = []
            acc_scores = []
            f1_scores  = [] 
            
            for fold, (train_index, test_index) in enumerate(kf.split(X)):
                X_train, X_test = X.iloc[train_index], X.iloc[test_index]
                y_train, y_test = y[train_index], y[test_index]
            
                X_test, y_test = original_test_dataset_transform(df_for_test_scaled, test_columns)
            
                oversample = SMOTE(random_state=2)
                X_train_res, y_train_res = oversample.fit_resample(X_train, y_train)
            
                clf_resample = lgb.LGBMClassifier(
                    boosting_type='gbdt',
                    num_leaves=50, reg_alpha=0.0, reg_lambda=1,
                    max_depth=-1, n_estimators=100, objective='binary',
                    subsample=0.7, colsample_bytree=0.7, subsample_freq=1,
                    learning_rate=0.05, min_child_weight=50, random_state=2000, n_jobs=100, #verbose=-1
                )
            
                clf_resample.fit(X_train_res, y_train_res,
                                 eval_set=[(X_train_res, y_train_res)],
                                 eval_metric='auc', early_stopping_rounds=1000, 
                                 # verbose=-1
                                )
            
                model_explanation[data_set + "_smote_fold_" + str(fold+1)] = clf_resample
            
                y_pred = clf_resample.predict_proba(X_test)[:, 1]
            
                y_demonstrate = {}
                y_demonstrate["y_pred"] = y_pred
                y_demonstrate["y_test"] = y_test
            
                key_words = "Smote_LightGBM_" + data_set + "_fold_" + str(fold+1)
            
                fpr_gbc, tpr_gbc, thresh_gbc = roc_curve(y_test, y_pred)
                
                auc = roc_auc_score(y_test, y_pred)
                auc_scores.append(auc)

                acc = accuracy_score(y_test, output_y_pred(y_pred))
                acc_scores.append(acc)

                f1  = f1_score(y_test,output_y_pred(y_pred))
                f1_scores.append(f1)
            
                print(f"{key_words} train AUC score: {auc}")
            
            avg_auc = np.mean(auc_scores)
            std_auc = np.std(auc_scores)

            avg_acc = np.mean(acc_scores)
            std_acc = np.std(acc_scores)

            avg_f1 = np.mean(f1_scores)
            std_f1 = np.std(f1_scores)
            
            print(f"Average AUC score across folds: {avg_auc}")
            print(f"Standard deviation of AUC scores: {std_auc}")
            
            algorithms_performances["Smote_LightGBM_" + data_set + "_avg_auc"] = avg_auc

            performances_metrics(algorithms_performances,
                                     avg_auc,std_auc,
                                     avg_acc,std_acc,
                                     avg_f1,std_f1,
                                     key_words,
                                     y_test, y_pred)

            pic_name = "Smote_LightGBM" + " " +  str(round(avg_auc,3))

            pic_name_list.append(pic_name)
            fpr_list.append(fpr_gbc)
            tpr_list.append(tpr_gbc)

        plot_roc_curve(fpr_list, tpr_list,pic_name_list)

In [ ]:
stats_plot_data_all = pd.DataFrame(algorithms_performances,index=["AUC","AUC std",
                                                                  "Accuracy","Accuracy std",
                                                                  "F1 Score","F1 Score std",
                                                                  "Precision","Recall",
                                            "KS","G_Mean","BAcc","tpr","MCC",
                                            "counts_ratio","kappa"]).transpose()

stats_plot_data_all.to_excel("/kaggle/working/stats_plot_data_all.xlsx")
# pd.set_option('display.max_rows', None) 
stats_plot_data_all

In [ ]:
stats_plot_data_all.to_excel("/kaggle/working/stats_plot_data_all.xlsx")
# to_csv('/kaggle/working/synthetic_2000.csv')

In [ ]:
# if key_words not in auc_scores.keys():
#     auc_scores[key_words]=[auc]
# else:
#     auc_scores[key_words].append(auc)
    
# pd.DataFrame(performances_metrics(algorithms_performances_new,key_words,y_test, y_pred)).transpose()

In [ ]:
# df_labelled_dict.keys()
df_labelled_dict = df_dataset_dict

In [ ]:
n_rows = len(df_labelled_dict)
print("n_rows",n_rows)

dataset_name_indexes = range(len(df_labelled_dict.keys()))
print("dataset_name_indexes",len(dataset_name_indexes))

int(len(dataset_name_indexes)/n_rows)

In [ ]:
def datasets_dict_with_support_sizes_plot(df_labelled_dict):
    
    
    palette = {'0': 'orange', '1': '#067ef7'}
    
    n_rows = int(len(df_labelled_dict)/4)
    
    dataset_name_indexes = range(len(df_labelled_dict.keys()))
                               
    n_cols = int(len(dataset_name_indexes)/n_rows)
                               
    sns.set_theme(style="white")
    
    n = len(df_labelled_dict)
    
    fig, ax = plt.subplots(n_rows,int(n_cols), sharey="row",figsize=(10,10))
    print("@@@ ",n_rows,int(n_cols))

    ck = 0
    keys = list(df_labelled_dict.keys())
    
    for dataset_name_index in dataset_name_indexes:
        
        
        X_train = df_labelled_dict[keys[dataset_name_index]]
        y_train =  df_y[keys[dataset_name_index]]#.to_numpy()
     
        X_stand_original = StandardScaler()   .fit_transform(X_train) 
        pca_original     = PCA(n_components=2).fit(X_stand_original)
        X_pca            = pca_original       .transform(X_stand_original)
        
        
        jk = dataset_name_index%n_cols
        
        print("== ",ck//n_cols,jk)
 
        sns.scatterplot(x         = X_pca[:, 0], 
                        y         = X_pca[:, 1], 
                        hue       = y_train.astype(int).astype(str), 
                        alpha     = 0.3, 
                        ax        = ax[ck//n_cols,jk],
                        edgecolor = "white",
                        palette   = palette,
                        edgecolors=None,
                        sizes     = (70, 400),
                        s         = 100)

#         print("locations : ",[ck//n_cols,jk])
        
        dataset_name_key = keys[dataset_name_index]

        ax[ck//n_cols,jk].title.set_text(dataset_name_key)
        ax[ck//n_cols,jk].grid(False)
        ax[ck // n_cols, jk].set_xlim(-15, 15)
        ax[ck // n_cols, jk].set_ylim(-5, 15)
        # ax[ck // n_cols, jk].set_xticks(range(-15, 16, 5))
        ax[ck // n_cols, jk].set_yticks(range(-10, 16, 5)) # Set y-ticks for every 5 units


        ck += 1

    plt.tick_params(bottom=False, labelbottom=False, labelleft=False)
    
    plt.tight_layout()
    plt.tight_layout(pad=2.0)
    plt.grid(False)
#     algos = str("algorithms == " + algo)
#     plt.title(algos)
    plt.savefig("/kaggle/working/data distribution plotting.png",dpi=600)
    plt.tight_layout()
    plt.show()
    
datasets_dict_with_support_sizes_plot(df_labelled_dict)

In [ ]:
# Obtain gain feature importance
keys  = list(df_labelled_dict.keys())

print(keys)
model = model_explanation[keys[0]+"_fold_1"]

# df_labelled_x = X_train_original

gain_importance = model.feature_importances_

# Display feature importance with feature names
feature_names = df_labelled_x.columns
gain_importance_df = pd.DataFrame({'Feature': feature_names, 'Target': gain_importance})
print(gain_importance_df.sort_values(by='Target', ascending=False))

(pd.Series(gain_importance, index=feature_names).nlargest(20).plot(kind='barh',label="importance"))

In [ ]:
import shap
import numpy as np

# model_explanation

In [ ]:
df_labelled_dict.keys()

In [ ]:
from matplotlib import pyplot as plt

n = 0

scenarios = keys[n]
print("scenarios : == ",scenarios)
model = model_explanation[scenarios +"_fold_1"]


print(scenarios)

#==========================================

explainer = shap.TreeExplainer(model)


shap.initjs() 


df_labelled_dict = df_dataset_dict

shap_values = explainer.shap_values(df_labelled_dict[scenarios])


plt.figure(figsize=(10, 6))

plot_shap = shap.summary_plot(  shap_values[1], 
                                df_labelled_dict[scenarios],
                                show=False, 
                                max_display = 20,
#                                 max_display=df_labelled_dict[scenarios].shape[1]
                             )

plt.xlim(-2, 2)
plt.title(scenarios, fontsize = 20)


plt.savefig("/kaggle/working/global shap, complete dataset.png",dpi=600)
plt.show()

# plt.savefig(plot_shap)



In [ ]:
df_for_train.label.unique()

In [ ]:
df_shap_dict.keys()
df_shap_dict["complete data_fold_1"]

In [ ]:
force_plot_dict = {} 
force_plot_dict["Graduate" ] = 0
force_plot_dict["Dropout"  ] = 4
# force_plot_dict["Enrolled" ] =2

In [ ]:
# "/kaggle/working/"+ status

# plt.savefig("/kaggle/working/global shap, without academic dataset.png",dpi=600)

In [ ]:
explainer.expected_value
shap_values[0][1]

In [ ]:
n = 0

#==============================

# status ="/kaggle/working/individual shap _" + keys[n] +  "plot2_Enrolled_.png"
# index = force_plot_dict["Enrolled" ] 
# print ( keys[n] + "\n" + "the Enrolled student...")

# shap.force_plot(np.around(explainer.expected_value[1], decimals = 2), 
#                 np.around(shap_values[1][index,:], decimals = 2),
#                 np.around(df_labelled_dict[keys[n]].iloc[index,:], decimals = 2),
#                 # link="logit",
#                 text_rotation=15,  
#                 matplotlib=True,
#                 show=False
#                )
# # plt.xlim(-3, 3)
# plt.savefig(status,dpi=600)
# # plt.show()

#==============================

status = "/kaggle/working/individual shap _" + keys[n] + "plot2_Graduate_.png"
index = force_plot_dict["Graduate"] 
print ( keys[n] + "\n" + "the Graduate student...")
shap.force_plot(np.around(explainer.expected_value[1], decimals = 2), 
                np.around(shap_values[1][index], decimals = 2), 
                np.around(df_labelled_dict[keys[n]].iloc[index], decimals = 2),
                matplotlib=True,show=False,
                text_rotation=15,  
               )
# plt.xlim(-3, 3)
plt.savefig(status,dpi=600)
plt.show()


#==============================

status = "/kaggle/working/individual shap _" + keys[n] + "plot2_Dropout_.png"
index = force_plot_dict["Dropout" ] 
print ( keys[n] + "\n" +"the dropout student...")
shap.force_plot(np.around(explainer.expected_value[1], decimals = 2), 
                np.around(shap_values[1][index], decimals = 2), 
                np.around(df_labelled_dict[keys[n]].iloc[index], decimals = 2),
                matplotlib=True,show=False,
                text_rotation=15,  
               )


# plt.title(status, fontsize = 20,loc='left')
# plt.xlim(-3, 3)
plt.savefig(status,dpi=600)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

n = 2
scenarios = keys[n]

model = model_explanation[scenarios+"_fold_1"]
print("scenarios : == ",scenarios)

explainer = shap.TreeExplainer(model)
shap.initjs() 

shap_values = explainer.shap_values(df_labelled_dict[scenarios])


plt.figure(figsize=(10, 6))


shap.summary_plot(  shap_values[1], 
                    df_labelled_dict[scenarios],
                    show=False, 
                      max_display = 20,
#                     max_display=df_labelled_dict[scenarios].shape[1]
                 )



plt.xlim(-2, 2)
plt.title(scenarios, fontsize = 20)

plt.title(scenarios, fontsize=20)
plt.savefig("/kaggle/working/global shap, without academic dataset.png",dpi=600)


plt.show()


# plt.savefig(plot_shap)
